In [11]:
import torch
from IPython.display import Image, clear_output, display
from ipywidgets import widgets
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from subprocess import run
import easyocr
import warnings

warnings.filterwarnings(action="ignore")

In [14]:
def infer(img_path):
    results = run(["python3","detect.py" ,"--weights", "runs/train/chkpt/best.pt", "--source", img_path, "--exist-ok", "--save-txt", "--save-conf"], capture_output=True)
    return results

def extractText(img):
    reader = easyocr.Reader(['en'], gpu=False)
    text = " ".join([li[1] for li in reader.readtext(img)]) 
    return text

## Detecting **Person, Logo, Cartoon and Text**

In [16]:
img_path = "test_data/temp_img.jpg"
btn = widgets.Button(description="Get Results")
uploader = widgets.FileUpload(accept='image/*', multiple=False)
image_widget = widgets.Image(width=300,height=400)
image_widget.visible = False


out = widgets.Output()
def on_btn_clicked(b):
    with out:
        
        clear_output()
        img = uploader.data[0]
        uploader.data = []
        with open("test_data/temp_img.jpg", 'wb') as f:
            f.write(img)
        
        label_path = "runs/detect/exp/labels/temp_img.txt"
        if os.path.exists(label_path):
            os.remove(label_path)
        
        result = infer(img_path)
        text = extractText(img)
        
        line = ""
        cat_idxs = [] 
        categories = ['Cartoon', 'Person', 'Logo']
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                line = f.readline().strip()
                while line:    
                    parts = line.split()
                    cat_idxs.append(int(parts[0]))
                    line = f.readline().strip()
        
        nrof_person = cat_idxs.count(1)
        nrof_cartoon = cat_idxs.count(0)
        nrof_logo = cat_idxs.count(2)


        image_widget.visible = True
        image_widget.value = open("./runs/detect/exp/temp_img.jpg", 'rb').read()
        image_widget.format="jpg"

        print(f"Image contains:\nNo of Cartoon:{nrof_cartoon}\t No of Person:{nrof_person}\t\tNo of Logo:{nrof_logo}\n\nText in image:\n{text}")
        
btn.on_click(on_btn_clicked)
vbox = widgets.VBox([widgets.HBox([uploader, btn]), image_widget], width=300,height=400)
widgets.HBox([vbox, out])